In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
curveballDL = pd.read_csv('DraftLeagueCurveball.csv')
curveballMLB = pd.read_csv('MLBcurveball.csv')

In [3]:
curveballDL = curveballDL.astype(str)
curveballMLB = curveballMLB.astype(str)

In [4]:
curveballDL['league'] = 'Draft League'
curveballMLB['league'] = 'MLB'

In [5]:
pitchersDL = list(set(curveballDL['pitcher']))

In [6]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [7]:
def get_index_from_pitcher(pitcher):
    return curveball[curveball.pitcher == pitcher].index[0]

In [8]:
def get_pitcher_from_index(index):
    return curveball[curveball.index == index].values[0][0]

In [9]:
mlb_draft_pitcher = []
mlb_comp = []
cos_value = []

for x in pitchersDL:
    
    mlb_draft_pitcher.append(x)
    
    try:

        curveball = pd.concat([curveballDL, curveballMLB])

        curveball["combined_features"] = curveball.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(curveball["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        curveball = curveball[(curveball['pitcher'] == pitcher_like) | (curveball['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        mlb_comp.append(get_pitcher_from_index(sorted_similar_pitchers[1][0]))
        
        cos_value.append(sorted_similar_pitchers[1][1])
                
    except IndexError:
        mlb_comp.append('No Comp')
        cos_value.append('No Similarity')
        pass

In [10]:
cosine_comp = pd.DataFrame(list(zip(mlb_draft_pitcher, mlb_comp, cos_value)), columns=['MLB DL Pitcher', 'MLB Comp', "Cosine Value"])
cosine_comp

,MLB DL Pitcher,MLB Comp,Cosine Value
0,"Wilson, Jeffrey","Ryan, Kyle",0.272166
1,"McMahon, Ian","Bourque, James",0.251976
2,"Blatter, Brock","Slater, Austin",0.272166
3,"Rodriguez, Joe Joe",No Comp,No Similarity
4,"Heine, Dylan",No Comp,No Similarity
...,...,...,...
126,"Freeberger, Dominic","Lauer, Eric",0.316228
127,"Nett, Braden","Fulmer, Carson",0.3
128,"Pryor, Luke","Kluber, Corey",0.251976
129,"Brown, Matt","Cotton, Jharel",0.33541


In [11]:
cosine_comp.to_csv('curveball_cosine_comp.csv', sep='\t')